In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Navigate to the directory where your file is located
import os
os.chdir('/content/drive/MyDrive/Data MSc thesis')

# Read the CSV file
data = pd.read_csv("Preprocessed_15min_2023.csv")

In [ ]:
import pandas as pd

# Assuming 'year' is a column in your DataFrame 'data'
data_year_encoded = pd.get_dummies(data, columns=['year'])


In [ ]:
import pandas as pd

# Assuming 'month' is a column in your DataFrame 'data_year_encoded'
# Map the month numbers to month names
month_names = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
               7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}

# Map month numbers to month names and then one-hot encode
data_month_encoded = pd.get_dummies(data_year_encoded, columns=['month'])
data_month_encoded.rename(columns=month_names, inplace=True)


In [ ]:
data_day_encoded = pd.get_dummies(data_month_encoded, columns=['day'])
#data_month_encoded.rename(columns=month_names, inplace=True)


In [ ]:
data_dayofweek_encoded = pd.get_dummies(data_day_encoded, columns=['day_of_week'])
#data_month_encoded.rename(columns=month_names, inplace=True)


In [ ]:
# Assuming 'weekend' is the column containing integer values (0 and 1)
data_dayofweek_encoded['weekend'] = data_dayofweek_encoded['weekend'].astype(bool)
data_dayofweek_encoded['is_public_holiday'] = data_dayofweek_encoded['is_public_holiday'].astype(bool)


In [ ]:
data_hour_encoded = pd.get_dummies(data_dayofweek_encoded, columns=['hour'])
#data_month_encoded.rename(columns=month_names, inplace=True)


In [ ]:
data_minute_encoded = pd.get_dummies(data_hour_encoded, columns=['minute'])
#data_month_encoded.rename(columns=month_names, inplace=True)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Assuming 'intensiteit' is the column you want to standardize
intensiteit_values = data_minute_encoded['intensiteit'].values.reshape(-1, 1)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the intensiteit values and transform them
intensiteit_scaled = scaler.fit_transform(intensiteit_values)

# Replace the 'intensiteit' column with the standardized values
data_minute_encoded['intensiteit'] = intensiteit_scaled


In [ ]:
# Sort the data by id_meetlocatie and date
data1 = data_minute_encoded.sort_values(by=["id_meetlocatie", "date"])

# Group by id_meetlocatie and add the lagged feature
data1["avg_speed_t-1"] = data1.groupby("id_meetlocatie")["avg_speed"].shift(1)

In [ ]:
# Sort the data by id_meetlocatie and date
data2 = data1.sort_values(by=["id_meetlocatie", "date"])

# Group by id_meetlocatie and add the lagged feature
data2["avg_speed_t-2"] = data2.groupby("id_meetlocatie")["avg_speed_t-1"].shift(1)

In [ ]:
# Sort the data by id_meetlocatie and date
data3 = data2.sort_values(by=["id_meetlocatie", "date"])

# Group by id_meetlocatie and add the lagged feature
data3["avg_speed_t-3"] = data3.groupby("id_meetlocatie")["avg_speed_t-2"].shift(1)

In [ ]:
data4 = data3.dropna(subset=['avg_speed_t-1', 'avg_speed_t-2', 'avg_speed_t-3'])

In [ ]:
from sklearn.preprocessing import StandardScaler

# Columns to standardize
columns_to_standardize = ['avg_speed_t-1', 'avg_speed_t-2', 'avg_speed_t-3']

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the selected columns and transform them
data4[columns_to_standardize] = scaler.fit_transform(data4[columns_to_standardize])


<ipython-input-41-94757408da41>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4[columns_to_standardize] = scaler.fit_transform(data4[columns_to_standardize])


In [ ]:
encoded_data = pd.get_dummies(data4, columns=['id_meetlocatie'])

In [ ]:
#2016-2019
# vanaf 15 maart 2019 test split
# 19 oktober 2018 val split
# tot 19 oktober 2018 train split

#2023
#vanaf 20 oktober test split
#tot en met 12 september train split
#tussen in val split

In [ ]:
import pandas as pd

# Assuming 'data' is your DataFrame containing the dataset

# Convert 'date' column to datetime type
encoded_data['date'] = pd.to_datetime(encoded_data['date'])

# Perform train-validation-test split based on date
train_data = encoded_data[encoded_data['date'] < '2023-09-13']
test_data = encoded_data[encoded_data['date'] > '2023-10-19']
val_data = encoded_data[(encoded_data['date'] >= '2023-09-13') & (encoded_data['date'] <= '2023-10-19')]

# Optionally, you may reset the index of each split DataFrame
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)


In [ ]:
train_data = train_data.drop(columns=['date'])
val_data = val_data.drop(columns=['date'])
test_data = test_data.drop(columns=['date'])

In [ ]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
# See shape to find number of rows

In [ ]:
import pandas as pd

# Assuming train_data, val_data, and test_data are pandas DataFrames
train_data = train_data.dropna()
val_data = val_data.dropna()
test_data = test_data.dropna()

print("Number of instances in train_data after removing NaN values:", len(train_data))
print("Number of instances in val_data after removing NaN values:", len(val_data))
print("Number of instances in test_data after removing NaN values:", len(test_data))

Number of instances in train_data after removing NaN values: 533892
Number of instances in val_data after removing NaN values: 77528
Number of instances in test_data after removing NaN values: 154132


In [ ]:
train_data.shape

(533892, 109)

In [ ]:
val_data.shape

(77528, 109)

In [ ]:
test_data.shape

(154132, 109)

In [ ]:
train_data = train_data.iloc[:533890]
val_data = val_data.iloc[:77525]
test_data = test_data.iloc[:154130]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
from kerastuner.tuners import RandomSearch

# Set random seed for TensorFlow
tf.random.set_seed(42)

# Set random seed for numpy operations
np.random.seed(42)

# Define the features and target variable
X_train = train_data.drop(columns=['avg_speed']).values
y_train = train_data['avg_speed'].values

X_val = val_data.drop(columns=['avg_speed']).values
y_val = val_data['avg_speed'].values

X_test = test_data.drop(columns=['avg_speed']).values
y_test = test_data['avg_speed'].values

# Convert X_train to float32
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')

X_val = X_val.astype('float32')
y_val = y_val.astype('float32')

X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

# Reshape data for LSTM input: (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0] // 5, 5, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0] // 5, 5, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0] // 5, 5, X_test.shape[1]))

# Reshape target variables
y_train = y_train.reshape((y_train.shape[0] // 5, 5))
y_val = y_val.reshape((y_val.shape[0] // 5, 5))
y_test = y_test.reshape((y_test.shape[0] // 5, 5))

def build_model(hp):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=hp.Int('units', min_value=192, max_value=256, step=32),
                                  input_shape=(X_train.shape[1], X_train.shape[2]))))
    model.add(Dense(224, activation='relu'))
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.1, max_value=0.3, step=0.1)))
    model.add(Dense(5))

    # Define the learning rate parameter
    hp_learning_rate = hp.Choice('learning_rate', values=[3e-4])

    # Compile the model with the specified learning rate
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='mean_squared_error')

    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=3,
    executions_per_trial=1,
    directory='hyperparameter_tuning',
    project_name='bidirectional_lstm_hyperparameter_tuning_1min_2016_2019v7_3'
)

tuner.search(X_train, y_train, epochs=60, batch_size=75, validation_data=(X_val, y_val), callbacks=[EarlyStopping(patience=4)])

best_model = tuner.get_best_models(num_models=1)[0]

# Print the hyperparameters used in the best model
print("Best hyperparameters:")
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hyperparameters.values)

# Predict on the validation set for hyperparameter tuning
y_val_pred = best_model.predict(X_val).reshape((-1,))

# Calculate RMSE and MAE on the validation set
rmse_val = np.sqrt(mean_squared_error(y_val.reshape((-1,)), y_val_pred))
mae_val = mean_absolute_error(y_val.reshape((-1,)), y_val_pred)

print("Validation set metrics:")
print("RMSE:", rmse_val)
print("MAE:", mae_val)

# Predict on the test set
y_test_pred = best_model.predict(X_test).reshape((-1,))

# Calculate RMSE and MAE on the test set
rmse_test = np.sqrt(mean_squared_error(y_test.reshape((-1,)), y_test_pred))
mae_test = mean_absolute_error(y_test.reshape((-1,)), y_test_pred)

print("\nTest set metrics:")
print("RMSE:", rmse_test)
print("MAE:", mae_test)

# Append predictions to validation and test data
#val_data['predictions_lstm_15min_2016_2019'] = y_val_pred
test_data['predictions_lstm_15min_2023'] = y_test_pred

# Save predictions to new dataframes
#predictions_val_lstm_15min = val_data.copy()
predictions_test_lstm_15min = test_data.copy()

# Optionally, you can save these dataframes to CSV files
#predictions_val_lstm_15min.to_csv('predictions_val_lstm_1min_2016_2019.csv', index=False)
predictions_test_lstm_15min.to_csv('predictions_test_lstm_15min_2023_new.csv', index=False)

Trial 3 Complete [00h 03m 08s]
val_loss: 6.315047740936279

Best val_loss So Far: 5.82101583480835
Total elapsed time: 00h 12m 57s
Best hyperparameters:
{'units': 192, 'dropout_rate': 0.2, 'learning_rate': 0.0003}
485/485 [==============================] - 3s 2ms/step
Validation set metrics:
RMSE: 2.4126782
MAE: 0.91061616
964/964 [==============================] - 2s 2ms/step

Test set metrics:
RMSE: 2.7711346
MAE: 1.0795212
